### OC spatial structure

- 2D, 3D structure
- Model fitting
- tidal, core radii


In [2]:
using PyCall
using Distributions, Statistics
using DataFrames
using Distances, LightGraphs, SimpleWeightedGraphs
using Random , Printf, Glob
import PyPlot , CSV

rootdir = "/home/stephane/Science/GAIA"

push!(LOAD_PATH,"$rootdir/master/src")
using GaiaClustering


@pyimport astroquery.vizier as vizier
@pyimport astropy.units as u
@pyimport astropy.coordinates as coord


## directory
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/test"
ocdir= "$wdir/oc"

cd(wdir)

In [3]:
## Test OC
##

#oc= CSV.read("$ocdir/NGC 6124-2.3deg-oc.csv" , delim= ";")
oc= CSV.read("$ocdir/NGC 7086-1.0deg-oc.csv", delim= ";")

,ra,dec,l,b,distance,pmra,pmdec,X,Y,Z,vl,vb,vrad,gbar,rp,bp
,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰
1,322.588,51.3556,94.2283,0.0521341,1672.01,-1.69564,-1.56564,1671.99,-5.89749,-3.76196,-18.2902,0.152185,NaN,15.0727,13.5945,14.6117
2,322.686,51.3391,94.2614,-0.00153599,1640.93,-1.66993,-1.61461,1640.91,-4.84059,-5.22912,-18.0648,-0.289149,NaN,16.3507,14.6722,16.0221
3,322.601,51.3638,94.2395,0.0527067,1642.32,-1.54933,-1.66081,1642.31,-5.47028,-3.67876,-17.6421,-1.17318,NaN,16.1403,14.509,15.7678
4,322.805,51.4962,94.4229,0.0624116,1693.41,-1.66808,-1.64562,1693.41,-0.220098,-3.50636,-18.8011,-0.520765,NaN,14.675,13.2838,14.1834
5,322.621,51.5978,94.4089,0.214773,1605.36,-1.87723,-1.61163,1605.36,-0.6029,0.944947,-18.8085,0.828554,NaN,15.3795,13.3523,14.3225
6,322.455,51.3924,94.1931,0.135711,1705.14,-1.61338,-1.49453,1705.13,-7.06308,-1.34923,-17.7743,0.15136,NaN,14.5613,13.2231,14.0487
7,322.598,51.6286,94.4197,0.246928,1749.06,-1.88835,-1.69633,1749.06,-0.327476,2.01112,-21.0396,0.460219,NaN,17.1825,15.5454,17.1161
8,322.668,51.5685,94.4105,0.173195,1720.84,-1.56654,-1.92731,1720.84,-0.598675,-0.235865,-20.073,-2.73675,NaN,18.5067,16.7454,18.5308
9,322.574,51.6506,94.4239,0.273015,1691.94,-1.6225,-1.7202,1691.94,-0.190436,2.71579,-18.9293,-1.14738,NaN,13.0162,11.622,12.5172
